In [4]:
import cv2
import numpy as np
import os

# Partie 1 : Charger les paramètres de calibrage
K = np.array([[676.51500784, 0, 337.679575],
              [0, 676.89640771, 209.73472297],
              [0, 0, 1]])
dist_coeffs = np.array([-4.98599586e-03, 3.23030615e-02, 5.47860434e-04, -4.93529845e-03, -8.59251548e-01])

# Partie 2 : Capture d'image et correction de la distorsion

# Fonction pour générer un nom de fichier unique
def generate_filename(directory='images', prefix='capture', extension='jpg'):
    if not os.path.exists(directory):
        os.makedirs(directory)
    counter = 1
    while True:
        filename = os.path.join(directory, f"{prefix}_{counter}.{extension}")
        if not os.path.exists(filename):
            return filename
        counter += 1

# Fonction pour corriger la distorsion de l'image
def undistort_image(image, K, dist_coeffs):
    h, w = image.shape[:2]
    new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(K, dist_coeffs, (w, h), 1, (w, h))
    undistorted_image = cv2.undistort(image, K, dist_coeffs, None, new_camera_matrix)
    # Recadrer l'image selon la région d'intérêt (roi)
    x, y, w, h = roi
    undistorted_image = undistorted_image[y:y+h, x:x+w]

    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(undistorted_image, cv2.COLOR_BGR2GRAY)
    # Appliquer un flou gaussien
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # Détecter les contours avec Canny
    edged = cv2.Canny(blurred, 50, 150)
    
   # Trouver les contours
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        # Supposons que l'objet le plus grand est celui d'intérêt
        contour = max(contours, key=cv2.contourArea)
        # Obtenir la boîte englobante de l'objet
        x, y, w, h = cv2.boundingRect(contour)
        # Dessiner la boîte englobante sur l'image undistortée
        cv2.rectangle(undistorted_image, (x, y), (x+w, y+h), (0, 255, 0), 2)

    return undistorted_image

# Partie principale : Capture de l'image et correction de la distorsion
def main():
    # Ouvrir une connexion à la webcam
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Erreur: Impossible d'ouvrir la webcam.")
        return

    while True:
        # Lire une frame de la webcam
        ret, frame = cap.read()

        if not ret:
            print("Erreur: Impossible de lire la frame de la webcam.")
            break

        # Afficher la frame
        cv2.imshow('Img', frame)

        # Attendre l'appui sur une touche
        key = cv2.waitKey(1) & 0xFF

        # Si la touche 's' est appuyée, capturer et enregistrer l'image
        if key == ord('s'):
            # Corriger la distorsion de l'image et détecter des objets
            undistorted_frame = undistort_image(frame, K, dist_coeffs)
            filename = generate_filename()
            cv2.imwrite(filename, undistorted_frame)
            print(f"Image capturée et enregistrée sous le nom '{filename}'.")
            # Afficher l'image undistortée avec la détection d'objet
            #cv2.imshow('Undistorted Image with Object Detection', undistorted_frame)
            #cv2.waitKey(0)  # Attendre une touche pour fermer
            break

    # Libérer la webcam et fermer les fenêtres
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Image capturée et enregistrée sous le nom 'images\capture_25.jpg'.
